In [2]:
# import torch
# import pandas as pd
import numpy as np
import scipy.io
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

2024-05-28 12:12:30.925648: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

ImportError: initialization failed

In [ ]:
# Load the .mat file
file_path = '../EV_Rank_1_52_RBs_50_UEs_1000_snaps.mat'
data = scipy.io.loadmat(file_path)

# Extract the relevant data
EV_data = data['EV_re_im_split']

# Check the shape and structure of the extracted data
data = EV_data
del EV_data
data.shape

In [ ]:
def create_sequences(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(data.shape[0]):  # iterate over samples
        for j in range(data.shape[1] - n_steps_in - n_steps_out + 1):  # iterate over timesteps
            seq_x = data[i, j:j + n_steps_in]
            seq_y = data[i, j + n_steps_in:j + n_steps_in + n_steps_out]
            X.append(seq_x)
            y.append(seq_y)
    return np.array(X), np.array(y)


In [ ]:

n_steps_in = 5
n_steps_out = 1

X, y = create_sequences(data, n_steps_in, n_steps_out)


In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
y = y.reshape((y.shape[0], y.shape[2]))

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
print("Training data shape:", X_train.shape, y_train.shape)
print("Validation data shape:", X_val.shape, y_val.shape)
print("Testing data shape:", X_test.shape, y_test.shape)


In [ ]:

# Define the LSTM model
model = Sequential()
model.add(LSTM(256, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train.shape[1]))  # Output layer


In [ ]:
#print the model



In [ ]:
# Compile the model
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')


In [ ]:
# Train the model
gpus = tf.config.experimental.list_physical_devices('GPU')

# print(tf.vis)
print(gpus)
# Fit the model
history = model.fit(X_train, y_train, epochs=100, batch_size=512, validation_data=(X_val, y_val), verbose=1)

# model.fit(X_train, y_train, epochs=100, batch_size=1024, validation_data=(X_val, y_val), verbose=2)


In [ ]:
# Make predictions on the test set
predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

# Calculate MSE
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error (MSE):", mse)

In [ ]:
# Calculate RMSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
y_test.shape

In [ ]:
predictions.shape

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
plt.figure(figsize=(15, 6))
plt.plot(y_test[20][50:200], label='True Values')
plt.plot(predictions[20][50:200], label='Predicted Values')
plt.title('True vs Predicted Values')
plt.xlabel('Timesteps')
plt.ylabel('Values')
plt.legend()
plt.show()


In [ ]:
import pandas
pandas.DataFrame(predictions[0][50:200], y_test[0][50:200])